In [ ]:
import requests
import gzip
import pandas as pd
import networkx as nx
import numpy as np

%matplotlib inline
import seaborn as sns

import ddot
from ddot import Ontology, get_gene_name_converter, parse_gaf, parse_obo, nx_to_NdexGraph, read_term_descriptions, ndex_to_sim_matrix
from ddot import expand_seed, melt_square, make_index, align_hierarchies, update_nx_with_alignment, parse_ndex_uuid
from ddot.utils import NdexGraph_to_nx
from ddot.utils import bubble_layout_nx

# Setup parameters

In [ ]:
alpha = 0.1
beta = 0.5

In [ ]:
similarity_uuid = 'd2dfa5cc-56de-11e7-a2e2-0660b7976219'
go_uuid = '8bfa8318-55ed-11e7-a2e2-0660b7976219'

In [ ]:
seed = ['FANCA','FANCB', 'FANCC', 'BRCA2', 'FANCD2', 'FANCE',
        'FANCF', 'FANCG', 'FANCI', 'BRIP1', 'FANCL', 'FANCM', 'PALB2', 'RAD51C', 'SLX4', 'ERCC4', 'RAD51',
        'BRCA1', 'UBE2T', 'XRCC2']

# Download similarity data from NDEx

In [ ]:
sim, sim_names = ndex_to_sim_matrix(
    similarity_uuid,
    ndex_server,
    ndex_user,
    ndex_pass,
    similarity='similarity',
    input_fmt='cx_matrix',
    output_fmt='matrix',
    subset=None)

# Expand known FA genes

In [ ]:
expand, expand_idx, sim_2_seed, fig = expand_seed(
    seed,
    sim,
    sim_names,
    agg='mean',
    min_sim=4,
    filter_perc=None,
    seed_perc=0,
    agg_perc=None,
    expand_size=None,
    figure=True,
)
expand = list(expand)
len(expand)

# Run CLIXO

In [ ]:
df_sq = pd.DataFrame(sim[expand_idx, :][:, expand_idx], index=expand, columns=expand)
df = melt_square(df_sq)

In [ ]:
ont = Ontology.run_clixo(df, alpha, beta)

# Align with Gene Ontology

## Read Gene Ontology from NDEx

In [ ]:
go_human = Ontology.from_ndex(go_uuid, ndex_server, ndex_user, ndex_pass)
go_human.summary()

## Mutually collapse ontologies

In [ ]:
ont_collapsed, go_human_collapsed = Ontology.mutual_collapse(ont, go_human)

## Do alignment

In [ ]:
alignment = align_hierarchies(ont_collapsed, go_human_collapsed, 3, 4)
alignment.head()

# Upload to NDEx

## Upload subnetworks supporting each term

In [ ]:
name = 'FanGO'
description = (
    'Data-driven ontology created by CLIXO '
    '(parameters: alpha={alpha}, beta={beta}). '
    'Created from similarity network '
    'at {ndex_server}/{ndex_uuid}').format(
        alpha=alpha,
        beta=beta,
        ndex_server=ndex_server,
        ndex_uuid=similarity_uuid)

In [ ]:
term_2_uuid = ont.upload_subnets_ndex(
    df,
    ['similarity'],
    ndex_server,
    ndex_user,
    ndex_pass,
    name,
    propagate=True,
    public=True,
    verbose=True
)

## Upload ontology

In [ ]:
ont_ndex = ont.to_NdexGraph(
    name=name,
    description=description,
    term_2_uuid=None,
    layout='bubble',
    represents=True)
update_nx_with_alignment(ont_ndex, alignment, use_node_name=True)

In [ ]:
ont_url = ont_ndex.upload_to(ndex_server, ndex_user, ndex_pass)
ont_uuid = parse_ndex_uuid(ont_url)
ont_url

In [ ]:
import ndex.client as nc
ndex = nc.Ndex(ndex_server, ndex_user, ndex_pass)
ndex.make_network_public(ont_uuid)